## Facial Age Estimator Example
In this simple example, we show how the [MAX-Facial-Age-Estimator](https://developer.ibm.com/exchanges/models/all/max-facial-age-estimator/) model can be used to detect human faces in an image and predict the ages of each face. Additionally, we show how to use the returned bounding box coordinates for each face to visualize the bounding boxes and the age predictions on the original input image.

## Setup
The notebook calls the MAX Facial Age Estimator microservice, which must be running.  You can either use the [hosted demo instance](http://max-facial-age-estimator.max.us-south.containers.appdomain.cloud), or follow the instructions for [deploying the microservice locally from the Dockerhub image](https://github.com/IBM/MAX-Facial-Age-Estimator#deploy-from-docker-hub).

In [ ]:
# This notebook requires matplotlib, Pillow and requests
# You only need to run the line below to install these if you don't already have them installed

! pip install -q matplotlib Pillow requests

In [ ]:
import io
from PIL import Image
import matplotlib
from matplotlib import pyplot as plt
import matplotlib.patches as patches
import requests

In [ ]:
# This url must point to a running instance of the model microservice
# By default this assumes you are using the hosted demo instance:

url = 'http://max-facial-age-estimator.max.us-south.containers.appdomain.cloud/model/predict'
# Comment the line above and uncomment the following line if you are running the model microservice locally
# url = 'http://localhost:5000/model/predict'

def call_model(input_img):
    """
    Takes in input image file path and generates face bboxes and ages.
    """
    files = {'image': ('image.jpg',open(input_img,'rb'), 'images/jpeg')}
    r = requests.post(url, files=files).json()
    
    return r

## Step 1: Visualizing the test image
First we load the image with Pillow and display the image in our notebook

In [ ]:
## First, load the image with Pillow and display the image in our notebook
img_path = 'assets/tom_cruise.jpg'
image = Image.open(img_path)
image

## Step 2: Call Model to detect faces and predict ages for each face

In [ ]:
# Call the facial age estimator model
model_response = call_model(img_path)

## Step 3: Visualize model response
The model returns JSON containing a predictions field which is an array of JSON objects, one for each face detected in the image. For each face, the bounding box coordinates are contained in the `detection_box` field, while the age estimated results are contained in the `age_estimation` field.

The bounding box coordinates are given in the format [xmin, ymin, width, height], i.e., height for y and width for x. 
We use these pixel coordinates to overlay the bounding boxes on the original image.

In [ ]:
# Explore the model results
import json
print(json.dumps(model_response, indent=2))

In [ ]:
# Get the image height and width
image_width, image_height = image.size
# Create figure and axes
fig, ax = plt.subplots()
# Set larger figure size
fig.set_dpi(600)
# Display the image
plt.imshow(image)

# Set up the color of the bounding boxes and age text
color = '#00FF00'
# For each face, draw the bounding box and predicted age
for prediction in model_response['predictions']:
    bbox = prediction['detection_box']    
    # Unpack the coordinate values
    y1, x1, y2, x2 = bbox
    # Map the normalized coordinates to pixel values: scale by image height for 'y' and image width for 'x'
    y1 *= image_height
    y2 *= image_height
    x1 *= image_width
    x2 *= image_width        
    age_label = 'Estimated age: {}'.format(prediction['age_estimation'])
    # Create the bounding box rectangle - we need the base point (x, y) and the width and height of the rectangle
    rectangle = patches.Rectangle((x1, y1), x2 - x1, y2 - y1, linewidth=1, edgecolor=color, facecolor='none')
    ax.add_patch(rectangle)
    # Plot the bounding box and the estimated age
    plt.text(x1, y1-5 , age_label, fontsize=4, color=color, fontweight='bold')
plt.axis('off')
plt.show()